In [237]:
import pandas as pd
df = pd.read_csv("data_full.csv", index_col=0)

In [238]:
df_wijk = pd.read_csv("Leefbaarometer-scores wijken 2002-2022.csv")
df_wijk=df_wijk.iloc[:,:4]
df_wijk=df_wijk[(df_wijk['jaar']==2020) | (df_wijk['jaar']==2022)]
df_wijk["wk_code"]=df_wijk["wk_code"].apply(lambda x: x[2:])
df_wijk.rename(columns={'wk_naam': 'name'}, inplace=True)
df_all = []
for i in range(0, len(df_wijk)-1, 2):
    row1 = df_wijk.iloc[i]
    row2 = df_wijk.iloc[i+1]
    
    # Calculate the average of the last column values
    average_value = (row1[-1] + row2[-1]) / 2
    
    # Create a new row with the average value
    new_row = [max(row1[0], row2[0]), 2021, average_value]
    
    # Append the new row to the averaged df_wijk
    df_all.append(new_row)


C:\Users\I6240624\AppData\Local\Temp\ipykernel_11004\444341776.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  average_value = (row1[-1] + row2[-1]) / 2
C:\Users\I6240624\AppData\Local\Temp\ipykernel_11004\444341776.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = [max(row1[0], row2[0]), 2021, average_value]


In [239]:
df_buurt = pd.read_csv("Leefbaarometer-scores buurten 2002-2022.csv")
df_buurt=df_buurt.iloc[:,:4]
df_buurt=df_buurt[(df_buurt['jaar']==2020) | (df_buurt['jaar']==2022)]
df_buurt["bu_code"]=df_buurt["bu_code"].apply(lambda x: x[2:])
df_buurt.rename(columns={'bu_naam': 'name'}, inplace=True)
for i in range(0, len(df_buurt)-1, 2):
    row1 = df_buurt.iloc[i]
    row2 = df_buurt.iloc[i+1]
    
    # Calculate the average of the last column values
    average_value = (row1[-1] + row2[-1]) / 2
    
    # Create a new row with the average value
    new_row = [max(row1[0], row2[0]), 2021, average_value]
    
    # Append the new row to the averaged df_wijk
    df_all.append(new_row)
df_all = pd.DataFrame(df_all, columns=["code", "year", "liveability_score"])
df_all["code"] = df_all["code"].astype(int)

C:\Users\I6240624\AppData\Local\Temp\ipykernel_11004\3285892393.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  average_value = (row1[-1] + row2[-1]) / 2
C:\Users\I6240624\AppData\Local\Temp\ipykernel_11004\3285892393.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_row = [max(row1[0], row2[0]), 2021, average_value]


In [240]:
df=pd.merge(df, df_all, left_on="gwb_code_10",right_on="code", how='inner')

In [241]:
# Define proportion function
def calculate_proportions(df, col1, col2, col3):
    df[col1] = df[col2] / df[col3]
    return df

# Define scoring function
def scoring(row, column):
    if row[column] > df[column].mean():
        return df[column].mean() / row[column]
    else:
        return row[column] / df[column].mean()

In [243]:
calculate_proportions(df, 'prop_0-14', 'Numbers.of.people.from.0.14.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_15-24', 'Number.of.people.from.15.24.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_25-44', 'Number.of.people.from.25.44.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_45-64', 'Number.of.people.from.45.64.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_65+', 'Number.of.people....65.years.old', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_low_ed', 'n_low_educated', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_med_ed', 'n_highschool_educated', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_high_ed', 'n_highly_educated', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_recipients', 'n_income_recipients', 'Number.of.inhabitants')
df['prop_employee']=((df["Number.of.inhabitants"]-df['n_income_recipients'])*(df['p_employees']/100))/df["Number.of.inhabitants"]
df["prop_self_employed"]=1-df['prop_employee']-df['prop_recipients']
df.dropna(inplace=True)

In [246]:
# Define df_score DataFrame
df_score = pd.DataFrame()

# Apply the function to each row in the 'prop_0-14' column
df_score['score_0-14'] = df.apply(scoring, column="prop_0-14", axis=1)
df_score['score_15-24'] = df.apply(scoring, column="prop_15-24", axis=1)
df_score['score_25-44'] = df.apply(scoring, column="prop_25-44", axis=1)
df_score['score_45-64'] = df.apply(scoring, column="prop_45-64", axis=1)
df_score['score_65+'] = df.apply(scoring, column="prop_65+", axis=1)
df_score['score_age']=(df_score['score_0-14']+df_score['score_15-24']+df_score['score_25-44']+df_score['score_45-64']+df_score['score_65+'])/5

In [249]:
calculate_proportions(df, 'prop_men', 'Number.of.man', 'Number.of.inhabitants')
calculate_proportions(df, 'prop_women', 'Number.of.women', 'Number.of.inhabitants')
df_score['score_men'] = df.apply(scoring, column="prop_men", axis=1)
df_score['score_women'] = df.apply(scoring, column="prop_women", axis=1)
df_score["score_gender"]= (df_score["score_men"]+df_score["score_women"])/2

df_score['score_household_size']=df.apply(scoring, column="Average.household.size", axis=1)
df_score['score_single_family']=df.apply(scoring, column='percent.of.single.family.housing', axis=1)
df_score['score_multi_family']=df.apply(scoring, column='percent.of.multifamily.housing', axis=1)
df_score["score_family"]= (df_score["score_single_family"]+df_score["score_multi_family"])/2


df_score['score_low_ed'] = df.apply(scoring, column="prop_low_ed", axis=1)
df_score['score_med_ed'] = df.apply(scoring, column="prop_med_ed", axis=1)
df_score['score_high_ed'] = df.apply(scoring, column="prop_high_ed", axis=1)
df_score["score_education"]= (df_score["score_low_ed"]+df_score["score_med_ed"]+df_score["score_high_ed"])/3


df_score['score_recipients'] = df.apply(scoring, column="prop_recipients", axis=1)
df_score['score_employee'] = df.apply(scoring, column="prop_employee", axis=1)
df_score['score_self_employed'] = df.apply(scoring, column="prop_self_employed", axis=1)
df_score['score_employment'] = (df_score['score_recipients']+df_score['score_employee']+df_score['score_self_employed'])/3

df_score['score_density'] = df.apply(scoring, column="Population.density", axis=1)
df_score['score_liveability'] = df['liveability_score']
df_score

,score_0-14,score_15-24,score_25-44,score_45-64,score_65+,score_age,score_men,score_women,score_gender,score_household_size,...,score_low_ed,score_med_ed,score_high_ed,score_education,score_recipients,score_employee,score_self_employed,score_employment,score_density,score_liveability
0,0.699629,0.634651,0.824498,0.826562,0.453134,0.687695,0.938554,0.936855,0.937704,0.816804,...,0.657047,0.997762,0.488939,0.714583,0.964374,0.856637,0.865676,0.895563,0.456962,4.113391
1,0.778692,0.677848,0.922371,0.992994,0.820230,0.838427,0.963432,0.968225,0.965828,0.932787,...,0.624403,0.882514,0.490206,0.665708,0.986666,0.969030,0.836921,0.930872,0.844079,3.965993
2,0.832406,0.646707,0.764417,0.928584,0.986382,0.831699,0.960659,0.961199,0.960929,0.867855,...,0.922777,0.872423,0.914622,0.903274,0.927653,0.748182,0.966961,0.880932,0.838583,4.035158
3,0.931519,0.948458,0.796237,0.761186,0.804705,0.848421,0.976032,0.977706,0.976869,0.890388,...,0.884313,0.753243,0.705013,0.780856,0.985228,0.974423,0.772484,0.910712,0.732152,4.031153
4,0.773576,0.959722,0.875506,0.763021,0.486361,0.771637,0.991092,0.990935,0.991013,0.783541,...,0.707304,0.745437,0.521130,0.657957,0.971146,0.851056,0.770482,0.864228,0.763355,3.966037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,0.641853,0.920547,0.903753,0.869190,0.841758,0.835420,0.966901,0.963537,0.965219,0.867855,...,0.543920,0.939078,0.613164,0.698721,0.991841,0.987349,0.732733,0.903974,0.107299,4.193052
589,0.693808,0.983086,0.744193,0.792643,0.727117,0.788169,0.949831,0.950469,0.950150,0.969955,...,0.695486,0.945055,0.750904,0.797148,0.969449,0.865538,0.915916,0.916968,0.412567,4.087528
590,0.817990,0.918267,0.805245,0.889083,0.706371,0.827391,0.968937,0.968809,0.968873,0.969955,...,0.842061,0.855772,0.754603,0.817478,0.962747,0.870149,0.751521,0.861472,0.492102,4.113782
591,0.906470,0.827183,0.923847,0.842782,0.899540,0.879965,0.991359,0.994566,0.992962,0.969955,...,0.705762,0.814629,0.683991,0.734794,0.961883,0.886638,0.641450,0.829990,0.609669,4.007289


In [253]:
df_score.to_csv("scores.csv")